In [1]:
import pandas as pd
import os

## Data cleansing
Calculate the adj price for OHLC

In [8]:
contracts = ['IF00', 'IH00', 'IC00']

adj_cols = [
    "S_DQ_ADJOPEN",
    "S_DQ_ADJHIGH",
    "S_DQ_ADJLOW",
    "S_DQ_ADJCLOSE",
]

preadj_cols = [
    "S_DQ_OPEN",
    "S_DQ_HIGH",
    "S_DQ_LOW",
    "S_DQ_CLOSE",
]

def calculate_adj_price(contracts):
    for contract in contracts:
        filepath = os.path.join(os.curdir, "1m_main_contracts", contract + '.csv')
        df = pd.read_csv(filepath)

        for pre_col, adj_col in zip(preadj_cols, adj_cols):
            df[adj_col] = round(df[pre_col] * df['S_DQ_ADJFACTOR'], 1)

    df.to_csv(filepath, index=False)

calculate_adj_price(contracts)

In [20]:
filepath = os.path.join(os.curdir, '1m_main_contracts', 'data.csv')
df = pd.read_csv(filepath)

In [21]:
contracts = df['S_INFO_CODE'].unique()

def sep_contracts():
    for contract in contracts:
        new_df = df[df['S_INFO_CODE'] == contract]
        new_df = new_df.reset_index(drop=True)
        new_df.to_csv(contract + '.csv', index=False)

In [8]:
df = pd.read_csv('1m_main_contracts/IH00.csv')

df

,S_INFO_CODE,TRADE_DT,S_DQ_OPEN,S_DQ_HIGH,S_DQ_LOW,S_DQ_CLOSE,S_DQ_PRESETTLE,S_DQ_ASK1,S_DQ_BID1,S_DQ_VOLUME,S_DQ_AMOUNT,S_DQ_OICHANGE,S_DQ_ADJFACTOR
0,IH00,2015-04-16 09:16:00,3100.0,3120.0,3089.0,3117.6,3058.8,3118.0,3117.6,602,560458860,393,1.0000
1,IH00,2015-04-16 09:17:00,3117.8,3167.8,3117.8,3156.0,3058.8,3157.2,3156.0,673,635091600,280,1.0000
2,IH00,2015-04-16 09:18:00,3156.4,3156.4,3121.4,3131.0,3058.8,3127.4,3127.0,652,614033460,285,1.0000
3,IH00,2015-04-16 09:19:00,3128.4,3132.2,3116.6,3121.4,3058.8,3122.0,3121.4,501,469339440,172,1.0000
4,IH00,2015-04-16 09:20:00,3122.0,3132.8,3122.0,3132.8,3058.8,3132.8,3132.4,436,409202580,169,1.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
396505,IH00,2021-12-21 14:56:00,3290.2,3291.0,3289.6,3289.8,3272.2,3290.2,3289.8,302,298108380,57,1.3603
396506,IH00,2021-12-21 14:57:00,3289.8,3290.4,3289.0,3290.2,3272.2,3290.4,3290.2,315,310873560,42,1.3603
396507,IH00,2021-12-21 14:58:00,3290.2,3290.8,3289.0,3289.0,3272.2,3289.2,3289.0,339,334574940,76,1.3603
396508,IH00,2021-12-21 14:59:00,3289.2,3289.4,3288.0,3288.8,3272.2,3289.0,3288.6,342,337439700,96,1.3603


# Extract trading time

In [2]:
contracts = ['IF00', 'IH00', 'IC00']

def open_df(contract):
    # filepath = os.path.join(os.curdir, '5m_main_contracts', contract + '.csv')
    filepath = os.path.join(os.curdir, '1m_main_contracts', contract + '.csv')
    df = pd.read_csv(filepath)
    time_df = df['TRADE_DT']
    
    return time_df

def group_time(time_df):
    time_df = time_df.str.split(pat=' ', expand=True)
    time_df = time_df.rename({0: 'date', 1: 'time'}, axis=1)

    group_time = time_df.groupby(by=time_df['date'])['time'].apply(list)
    group_time = group_time.to_frame()
    group_time.index = pd.to_datetime(group_time.index)

    return group_time

for contract_name in contracts:
    df = group_time(open_df(contract_name))
    save_path = os.path.join(os.curdir, 'trading_time', contract_name + '_1mtime.csv')
    df.to_csv(save_path)


In [4]:
def get_tradetime(today) -> list:
        """获取当日所有交易时间点"""
        # 读取当前合约对应的交易时间表
        filepath = os.path.join(os.curdir, 'trading_time', 'IF00_1mtime.csv')
        time_df = pd.read_csv(filepath, index_col='date')

        return time_df.loc[today.isoformat()]

In [7]:
import datetime
import ast
random_time = datetime.date(2018, 4, 16)
time_list = get_tradetime(random_time)
time_list = ast.literal_eval(time_list.values[0])
print(time_list)
print(len(time_list))

['09:31:00', '09:32:00', '09:33:00', '09:34:00', '09:35:00', '09:36:00', '09:37:00', '09:38:00', '09:39:00', '09:40:00', '09:41:00', '09:42:00', '09:43:00', '09:44:00', '09:45:00', '09:46:00', '09:47:00', '09:48:00', '09:49:00', '09:50:00', '09:51:00', '09:52:00', '09:53:00', '09:54:00', '09:55:00', '09:56:00', '09:57:00', '09:58:00', '09:59:00', '10:00:00', '10:01:00', '10:02:00', '10:03:00', '10:04:00', '10:05:00', '10:06:00', '10:07:00', '10:08:00', '10:09:00', '10:10:00', '10:11:00', '10:12:00', '10:13:00', '10:14:00', '10:15:00', '10:16:00', '10:17:00', '10:18:00', '10:19:00', '10:20:00', '10:21:00', '10:22:00', '10:23:00', '10:24:00', '10:25:00', '10:26:00', '10:27:00', '10:28:00', '10:29:00', '10:30:00', '10:31:00', '10:32:00', '10:33:00', '10:34:00', '10:35:00', '10:36:00', '10:37:00', '10:38:00', '10:39:00', '10:40:00', '10:41:00', '10:42:00', '10:43:00', '10:44:00', '10:45:00', '10:46:00', '10:47:00', '10:48:00', '10:49:00', '10:50:00', '10:51:00', '10:52:00', '10:53:00', '10